In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import numpy as np
import casadi as ca

In [2]:
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"
gravity = [0, 0, -9.81]
n_joints = alpha.get_n_joints(root, tip)

qddot_sym = alpha.get_forward_dynamics_crba(root, tip)
# ─── 2. Define CasADi symbols and build the ODE ─────────────────────────────
q     = ca.MX.sym('q',     n_joints)
q_dot = ca.MX.sym('q_dot', n_joints)
tau   = ca.MX.sym('tau',   n_joints)
dt    = ca.MX.sym('dt')
gravity    = ca.MX.sym('gravity')

k  = ca.MX.sym("k_", n_joints, 2)
viscous = ca.MX.sym('viscous', n_joints,2)
coulomb = ca.MX.sym('coulomb', n_joints,2)
I_Grotor  = ca.MX.sym("I_Grotor",    n_joints,2)             # rotor inertias
payload  = ca.MX.sym("payload")

q_ddot = qddot_sym(q, q_dot, tau,
                   gravity, k, viscous, coulomb, I_Grotor, payload)

# 2. state & RHS -------------------------------------------------------
x     = ca.vertcat(q, q_dot)                   # ← pure symbols
x_dot = ca.vertcat(q_dot, q_ddot) * dt    # RHS may be complex
p = ca.vertcat(
        dt,
        gravity,
        ca.reshape(k,   -1, 1), 
        ca.reshape(viscous,   -1, 1),        # (2*n_joints × 1)
        ca.reshape(coulomb,   -1, 1),
        ca.reshape(I_Grotor,  -1, 1),
        payload
    )
# 3. integrator --------------------------------------------------------
dae  = {'x': x, 'ode': x_dot, 'p': p, 'u': tau}
opts = {'simplify': True,
        'number_of_finite_elements': 200}      # default grid [0,1]

intg = ca.integrator('intg', 'rk', dae, 0, 1, opts)  # ◀ no extra 0,1

# next-state map
res    = intg(x0=x, u=tau, p=p)
x_next = res['xf']

px =ca.vertcat(
    ca.reshape(k,   -1, 1), 
    ca.reshape(viscous,   -1, 1),
    ca.reshape(coulomb,   -1, 1),
    ca.reshape(I_Grotor,  -1, 1)
        )
F_next_ = ca.Function('Mnext', [x, tau, dt, gravity, payload, px ], [x_next])
F_next_.save('arm.casadi')

In [3]:
# c , cpp or matlab code generation for forward dynamics
F_next_.generate("F_next_.c")
os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

0